In [19]:
import torch
from IPython import display 
from matplotlib import pyplot as plt
import numpy as np
import random
import torch.utils.data as Data
import torch.nn as nn
torch.manual_seed(1)  # 设置随机种子，使每次初始化数据相同
torch.set_default_tensor_type('torch.FloatTensor')

> 笔记：定义torch.set_default_tensor_type('torch.FloatTensor')可同时设置所有张量默认格式，避免各张量元素类型发生冲突

In [20]:
# 生成数据集
num_inputs = 2  # 输入项数
num_examples = 1000  # 样本数
true_w = [2, -3.4]  # 权值w
true_b = 4.2  # 偏差b
# 生成特征值
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
# 生成标签
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)
print(features[0], labels[0])

tensor([-0.0287, -0.6813]) tensor(6.4607)


In [22]:
# 读取数据
batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(
    dataset=dataset,  # torch tensor dataset format
    batch_size=batch_size,
    shuffle=True,
    # num_workers=2  # 多线程
    )
# 读取打印第一个小批量
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.6284, -0.0116],
        [ 0.0604, -0.3432],
        [-0.3272, -1.3504],
        [ 1.6368, -0.4355],
        [ 0.1573, -0.8141],
        [ 0.5641, -0.7605],
        [ 0.5901, -0.9844],
        [-0.6685,  0.1953],
        [-0.4084,  0.0398],
        [-0.8160,  1.4268]]) tensor([ 5.5058,  5.4863,  8.1324,  8.9355,  7.2805,  7.9032,  8.7157,  2.2158,
         3.2489, -2.2910])


In [23]:
# 定义模型
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward前向传播
    def forward(self, x):
        y = self.linear(x)
        return y
net = LinearNet(num_inputs)
print(net)  # 打印网络结构
# # 可用 nn.Sequential 更方便地搭建网络；有三种表示法
# # 表示法一
# net = nn.Sequential(nn.Linear(num_inputs, 1)'''此处可添加其他层''')
# 表示法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module...继续添加层
# # 表示法三
# from collection import OrderedDict
# net = nn.Sequential(OrderedDict([('linear', nn.Linear(num_inputs, 1)) '''添加其他层''' ]))
print(net)
print(net[0])

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [24]:
# 初始化模型参数
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)   # 可直接修改bias的data
# net[0].bias.data.fill_(0)

# 通过 net.parameters() 查看模型所有可学习参数，此函数返回一个生成器
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.0206, 0.0077]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


In [25]:
# 定义损失函数
loss = nn.MSELoss()

# 定义优化算法（SGD）
import torch.optim as optim
# 用于优化net所有参数的优化器实例，学习率设为0.03
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)  # 查看优化器参数

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [26]:
# 训练模型
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()  # 梯度清零
        l.backward()
        optimizer.step()
    print('epoch {}, loss: {}'.format(epoch, l.item()))

epoch 1, loss: 0.00015255835023708642
epoch 2, loss: 9.898704593069851e-05
epoch 3, loss: 2.7647271053865552e-05


In [ ]:
# 比较习得参数与真实参数
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

softmax回归笔记：https://www.kesci.com/org/boyuai/project/share/54b451b9329f1a38
多层感知机笔记：https://www.kesci.com/org/boyuai/project/share/fe09890d6a297909